## Imports

In [1]:
import os
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
from pysis import isis

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

/home/jmapel/.conda/envs/knoten/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


## Load Network and Generate Sensors

In [2]:
cubes = 'data/cubes.lis'
sensors = generate_sensors(cubes)

network = 'data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors) # autoseed did not generate ground points, calculate and repopulate the data frame

## Determine Which Sensor Parameters to Solve For

In [3]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

Image: MRO/CTX/1085197697:073
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1096561308:045
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1136952576:186
  

In [4]:
# Solve for angles and angular rates
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

## Compute the Column Indices for Parameters

In [9]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [11]:
num_observations = 2 * len(cnet)
W = np.eye(num_observations)
param_W = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

## Calculate Initial Sigma0

In [12]:
V = compute_residuals(cnet, sensors)
dof = num_observations - num_parameters
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)

print(sigma0)

4.204919720356099


## Populate Jacobian

In [13]:
J = compute_jacobian(cnet, sensors, solve_parameters, column_dict)

## Bundle Iteration

In [14]:
N = J.T.dot(W).dot(J) + param_W
C = J.T.dot(W).dot(V)
dX = np.linalg.inv(N).dot(C)

## Calculate Updated Sigma0

In [15]:
updated_sigma0 = np.sqrt((V.dot(W).dot(V) - dX.dot(J.T).dot(W).dot(V))/dof)
print(updated_sigma0)

0.8759391866290785


## Redundancy Number

In [ ]:
# redundancy for every measure
# vector will hold same order as the measures in the cnet df
# def compute_measure_redundancy
Qxx = np.linalg.inv(N)
Qvv = np.linalg.inv(W) - J.dot(Qxx).dot(J.T)
r = np.diagonal(Qvv.dot(W))

print(f'Minimum redundancy: {min(r)}')
print(f'Maximum redundancy: {max(r)}')
plt.boxplot(r)

## Data Snooping

In [ ]:
# For data snooping we need to calculate updated residuals
qvv = np.diagonal(Qvv)
sigma_vi = sigma0*np.sqrt(qvv)
wi = -V/sigma_vi
# k = 3.29 #critical values from Forstner
k = 4.1 #cricital value from Baarda

# plt.figure()
# plt.boxplot(wi)
plt.figure()
plt.hist(wi[wi < k], bins=np.linspace(-4,5,50))
plt.hist(wi[wi > k], bins=np.linspace(-4,5,50))

## Update Sensors and Ground Points

In [ ]:
# update the sensor partials
for sn, sensor in sensors.items():
    n_param = len(parameters[sn])
    for i in range(n_param):
        sensor.setParameterValue(i, dX[column_dict[sn]+i])
        
# update ground points
for _, row in cnet.iterrows():
    point_id = row['id']
    ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
    adj = dX[column_dict[point_id]:column_dict[point_id]+3] 
    cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

## Whole bundle process in a loop

In [29]:
sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
tol = 1e-5
##############

num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
column_dict = compute_coefficient_columns(cnet, sensors, parameters)
W = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dof = num_observations - num_parameters
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

# while abs(sigma0-old_sigma0) < tol:
max_iterations = 15
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
    N = J.T.dot(W).dot(J) + W_params # calculate the normal equation
    C = J.T.dot(W).dot(V) 
    dX = np.linalg.inv(N).dot(C) #calculate change in camera parameters and ground points
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    # update the sensor parameters
    for sn, sensor in sensors.items():
        n_param = len(parameters[sn])
        for i in range(n_param):
            old_param = sensor.getParameterValue(parameters[sn][i].index)
            sensor.setParameterValue(parameters[sn][i].index, old_param+dX[column_dict[sn][0]+i])

    # update ground points
    for _, row in cnet.iterrows():
        point_id = row['id']
        ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
        adj = dX[column_dict[point_id][0]:column_dict[point_id][1]]
        cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

    V = compute_residuals(cnet, sensors)
    sigma0 = np.sqrt((V.dot(W).dot(V) - )/dof)
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

iteration 0: sigma0 = 4.204919720356099

corrections: mean = 0.5972006200859573 min = -16.530186637502926 max = 38.517963751145366
iteration 1: sigma0 = 0.4798799074538198

corrections: mean = -0.3157749722902711 min = -4.4345688181259355 max = 4.523000328679814
iteration 2: sigma0 = 0.39384795864851363

corrections: mean = -0.2859252196376692 min = -1.141908099175438 max = 0.9061197893446893
iteration 3: sigma0 = 0.3871588619081049

corrections: mean = -0.2721494666435455 min = -0.8335001186655998 max = 0.667190895177844
iteration 4: sigma0 = 0.38552125886755745

corrections: mean = -0.26730251212387945 min = -0.8895901221295787 max = 0.4811660090911366
iteration 5: sigma0 = 0.3846843185729506

corrections: mean = -0.26511616014593453 min = -0.8951575560847784 max = 0.34103587869234825
iteration 6: sigma0 = 0.3841702166706269

corrections: mean = -0.26356775399369237 min = -0.8909058234004009 max = 0.2776169395638875
iteration 7: sigma0 = 0.38382380529992316

corrections: mean = -0.26